# Programming Assingnment 5.2 - LLM Agent with Hugging Face

이번에 수행할 과제는 LLM Agent 구현입니다. 5.1과 마찬가지로 Hugging Face 라이브러리를 사용할 예정이며, 이번에는 RAG(Retrieval Augmented Generation)의 기본 형태를 바탕으로 구현을 진행하게 됩니다. 즉, RAG Agent를 직접 구현해 보게 될 것입니다

![](https://huggingface.co/blog/assets/os_llms/thumbnail.png)

# Part 3. LLM Agent with Qwen

## 3-0. RAG이란 무엇인가요?

RAG는 LLM(Large Language Model)이 특정 콘텐츠를 학습 데이터에 포함하지 않거나, 이전에 학습한 내용이더라도 이를 제대로 인식하지 못하거나 잘못된 정보를 생성하는 문제를 해결하기 위한 접근 방식입니다.

만약 데이터가 정적이고 정기적으로 변경되지 않는 경우, 대규모 모델을 미세 조정하는 방법을 고려할 수 있습니다. 그러나 많은 경우, 미세 조정은 비용이 많이 들며, 반복적으로 수행할 경우 이전에 학습한 정보를 잃어버리는 catastrophic forgetting 문제가 발생할 수 있습니다.

외부 데이터는 별도의 임베딩 모델을 사용하여 임베딩 벡터로 변환되며, 이 벡터들은 데이터베이스에 저장됩니다. 임베딩 모델은 일반적으로 소형이기 때문에 임베딩 벡터를 정기적으로 업데이트하는 것이 모델을 미세 조정하는 것보다 빠르고, 저렴하며, 더 쉽습니다.

동시에, 미세 조정이 필요하지 않기 때문에, 더 강력한 LLM이 이용 가능해질 때 이를 교체하거나, 더 빠른 추론이 필요한 경우 더 작은 버전으로 전환할 수 있습니다.

다음 자료를 참고해주세요:
![](https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/rag-diagram.png)

![](https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/RAG_workflow.png)

---
**여러분들께서는 뉴스를 분석해주는 RAG Agent를 구현하셔야 됩니다.** 이 에이전트는 수집한 뉴스 데이터를 분석해 주요 정보를 추출하며, 사용자의 쿼리에 대한 답변도 제공합니다. 특히, 주식 투자와 같은 금융 활동에서 유용하게 활용될 수 있습니다. RAG Agent는 특정 기업이나 산업에 대한 최신 뉴스를 빠르게 분석하여, 투자에 필요한 핵심 정보를 제공합니다.

이번 과제를 통해 여러분들은 RAG 모델의 개념을 이해하고, 이를 활용해 사용자의 질문에 맞춰 뉴스를 검색하고 답변을 생성하는 시스템을 구축하게 됩니다. 과제의 목표는 사용자가 편리하게 이용할 수 있는 효율적인 뉴스 분석 에이전트를 완성하는 것입니다.

## 3-1. Pip install
필요한 패키지를 pip를 이용해서 설치합니다.

In [ ]:
!pip install datasets transformers sentence-transformers faiss-cpu accelerate

Device를 GPU로 바꿔줍니다.

다만, 코드를 구현하는 과정에서는 **CPU 사용**을 권장드립니다.

GPU가 가장 많이 필요한 단계는 학습하는 과정인데 구현하는 시간동안 colab의 GPU를 다 써버리면 막상 학습할 때 필요한 GPU 자원을 쓸 수 없게 됩니다. 따라서 코드를 모두 구현하고 train 코드가 잘 돌아가는지 확인한 뒤에 colab 상단 런타임 메뉴에서 런타임 유형을 GPU로 바꾼 뒤에 실행하면 됩니다.

In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('using device: ', device)

## 3-2. Prepare Dataset

아래 코드를 실행해서 데이터셋을 불러와 주세요. 여러분들께서 사용하실 데이터셋은
**[네이버 뉴스 기사 데이터셋](https://huggingface.co/datasets/daekeun-ml/naver-news-summarization-ko)** 입니다.

이 데이터셋은 2022년 7월 1일부터 2022년 7월 10일까지의 기간 동안 수집된 네이버 뉴스 기사들을 포함하고 있습니다. 데이터셋은 총 7개의 피처(feature)로 구성되어 있으며, 각 피처는 다음과 같은 정보를 제공합니다:

1. `date`: 기사가 발행된 날짜를 나타냅니다. 데이터셋에 포함된 모든 기사는 2022년 7월 1일부터 2022년 7월 10일까지의 기간에 발행된 것입니다.
2. `category`: 뉴스 기사의 카테고리를 나타냅니다. 이 데이터셋에서는 IT와 경제 분야의 기사들로 분류됩니다.
3. `press`: 해당 기사를 작성한 언론사의 이름입니다.
4. `title`: 뉴스 기사의 제목입니다. 기사의 주요 내용을 간략하게 나타냅니다.
5. `document`: 뉴스 기사의 본문입니다. 기사의 전체 내용을 포함하며, 분석에 필요한 핵심 텍스트 데이터입니다.
6. `link`: 해당 기사의 원본 URL입니다. 사용자는 이 링크를 통해 원문 기사를 확인할 수 있습니다.
7. `summary`: 기사의 주요 내용을 요약한 텍스트입니다. 기사의 핵심 정보를 빠르게 파악하는 데 도움을 줍니다.

`train`, `test`, `validation` 으로 구성되어 있는데 학습을 할게 아니라 전부 다 사용해봅시다. 실제 뉴스 데이터에는 요약본이 포함되어 있지 않으니, 해당 데이터를 제거하겠습니다. 뉴스 기사 데이터만 있다고 가정하고 agent를 구축해봅시다.

In [ ]:
from datasets import load_dataset, concatenate_datasets

# Load each split separately
train_ds = load_dataset("daekeun-ml/naver-news-summarization-ko", split="train")
test_ds = load_dataset("daekeun-ml/naver-news-summarization-ko", split="test")
val_ds = load_dataset("daekeun-ml/naver-news-summarization-ko", split="validation")

# Concatenate the datasets
dataset = concatenate_datasets([train_ds, test_ds, val_ds])
dataset = dataset.remove_columns(['summary'])

위에서 불러온 뉴스 데이터는 아래 자료에서 `Your knowledge base`에 해당하고 각각의 뉴스 기사가 `Documents`에 해당됩니다.

![](https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/RAG_workflow.png)

데이터가 어떻게 생겼는지 한번 볼까요?

In [ ]:
dataset[0]

## 3-3. Embeddings and Retriever

RAG(Retrieval-Augmented Generation) 모델에서 embedding과 retriever는 핵심적인 역할을 합니다. 이를 설명하기 위해 RAG의 기본 개념을 다시 한번 설명하겠습니다. RAG는 사용자가 입력한 질문에 대해 단순히 답변을 생성하는 것이 아니라, 관련 정보를 검색하고 이를 바탕으로 더 정확하고 풍부한 답변을 생성하는 방식입니다.

1. **Embedding**: Embedding은 텍스트를 고차원 벡터로 변환하는 과정입니다. 이 벡터는 텍스트의 의미를 숫자 공간에서 표현한 것으로, 유사한 의미를 가진 텍스트들은 서로 가까운 위치에 위치하게 됩니다. RAG에서 embedding을 사용하는 이유는, 검색 과정에서 유사한 내용을 효율적으로 찾기 위해서입니다. 사용자의 질문이나 문서의 의미를 수치화하여, 이를 기반으로 유사성을 판단하고, 관련 문서를 효과적으로 검색할 수 있게 됩니다. 각각의 document에 대한 embedding을 생성, 저장하고 이를 기반으로 검색을 하기 때문에 knowledge base를 **vector database** 라고 정의할 수 있습니다.

2. **Retriever**: Retriever는 embedding된 벡터들을 활용하여, 주어진 질문에 가장 관련성이 높은 문서들을 검색하는 역할을 합니다. RAG 모델은 단순히 답변을 생성하는 것보다, 실제로 관련성이 높은 정보를 바탕으로 답변을 생성하는 것이 훨씬 정확하고 유의미한 결과를 낳습니다. Retriever는 데이터베이스에서 질문과 가장 유사한 문서들을 찾아내고, 이를 모델에 제공하여 더 나은 답변을 생성할 수 있도록 돕습니다.


예를 들어, 사용자가 "Tesla의 최근 주가 변동 원인은 무엇인가요?"라는 질문을 RAG 모델에 입력했다고 가정해봅시다.

1. **Embedding**: 먼저, 사용자의 질문 "Tesla의 최근 주가 변동 원인은 무엇인가요?"는 embedding 과정을 통해 고차원 벡터로 변환됩니다. 이 벡터는 질문의 핵심 요소인 "Tesla", "주가", "변동", "원인" 등을 반영합니다.

2. **Retriever**: 그런 다음, retriever가 이 embedding 벡터를 기반으로 뉴스 데이터베이스에서 가장 관련성이 높은 문서들을 검색합니다. 예를 들어, "Tesla, 새로운 자율주행 기술 발표로 주가 급등", "Tesla, 2분기 실적 발표 후 주가 하락"과 같은 기사를 찾아냅니다.

3. **Generation**: RAG 모델은 이 검색된 문서들을 바탕으로 답변을 생성합니다. 예를 들어, "Tesla의 최근 주가 변동은 자율주행 기술 발표와 2분기 실적 발표가 주요 원인으로 작용했습니다."라는 답변이 생성될 수 있습니다.

이 예시에서 embedding은 사용자의 질문을 효과적으로 수치화하여 의미를 추출하고, retriever는 이 수치화를 바탕으로 관련 기사를 신속하게 찾아내어 모델이 신뢰할 수 있는 답변을 생성하도록 도와줍니다.

요약하자면, embedding은 텍스트를 의미 공간에서 수치화하여 유사성을 측정할 수 있도록 하고, retriever는 이 embedding을 바탕으로 관련 문서들을 검색하여 RAG 모델이 보다 정확한 답변을 생성할 수 있도록 지원합니다. 이 두 요소는 RAG의 강력한 검색 및 생성 기능을 실현하는 데 필수적입니다.

![](https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/RAG_workflow.png)


embedding을 만드는 방법은 여러 가지가 있지만 본 과제에서는 [sentence-transformer 라이브러리](https://sbert.net/)를 이용하겠습니다.
Sentence Transformer는 문장 수준에서의 의미를 효과적으로 벡터화하는 모델을 제공하는 라이브러리로, 문장의 의미를 고차원 벡터로 변환하는 데 매우 유용합니다. 이 라이브러리는 BERT, RoBERTa와 같은 강력한 언어 모델들을 기반으로 하며, 문장 간의 유사도를 계산하거나, 문서 내에서 의미적으로 유사한 문장들을 찾는 작업에 최적화되어 있습니다.

Sentence Transformer를 사용하면 문장이나 문서의 의미를 잘 보존하는 embedding을 생성할 수 있어, RAG 모델의 retriever 단계에서 정확한 정보 검색이 가능해집니다. 따라서 본 과제에서는 이 라이브러리를 활용해 사용자의 질문과 뉴스 데이터베이스 내 문서들을 embedding하고, 이를 기반으로 가장 관련성 높은 문서를 검색하여 더욱 정확한 답변을 생성할 것입니다.

아래 코드에서 documentation을 참고하여 sentence transformer를 한번 사용해보세요.

[Sentence Transformers Documentation](https://sbert.net/docs/sentence_transformer/usage/usage.html)

In [ ]:
from sentence_transformers import SentenceTransformer

ST = SentenceTransformer("jhgan/ko-sroberta-multitask")

# TODO: 문장 예시 각 3개 만들기
sentences1 = [

]

sentences2 = [

]

# TODO: 문장 임베딩 계산
embeddings1 = ST.
embeddings2 = ST.

# TODO: 코사인 유사도 계산
similarities = ST.

# 점수와 함께 쌍 출력
for idx_i, sentence1 in enumerate(sentences1):
    print(sentence1)
    for idx_j, sentence2 in enumerate(sentences2):
        print(f" - {sentence2: <30}: {similarities[idx_i][idx_j]:.4f}")

이제 데이터셋에 있는 각 document(뉴스 기사)에 대해 embedding을 생성해보세요. Embedding을 생성할 때는 기사 본문, 제목, 또는 둘 다 사용할 수 있으며, 필요에 따라 카테고리와 같은 추가 정보를 포함할 수도 있습니다. 어떤 정보를 활용할지는 자유롭게 결정하여 코드를 작성해주세요.

In [ ]:
def embed(batch):
    """
    Generate embeddings for each entry in the dataset.

    This function adds a new column called 'embeddings' to the dataset.

    Note:
    - You can modify the 'information' variable to combine multiple fields (e.g., 'title' and 'text')
      if needed for the embedding generation.
    """

    # TODO: Embedding 생성을 위한 정보 추출
    information =
    return {"embeddings": ST.encode(information)}  # Generate embeddings using the 'information' and return

# Apply the 'embed' function to the dataset in batches
dataset = dataset.map(embed, batched=True, batch_size=16)

[FAISS](https://github.com/facebookresearch/faiss)

**아래 코드에서 각각의 embedding에 대한 FAISS index를 생성해주세요. 왜 이 과정이 필요한지 자료를 참고하여 2-3줄로 간단하게 이유를 작성해주세요:**

In [ ]:
data = dataset.add_faiss_index("embeddings")

Query를 정의하고, 이를 기반으로 가장 가까운 K개의 문서를 검색하는 과정입니다. Query를 설정한 후, 결과를 확인해보세요.

In [ ]:
def search(query: str, k: int = 3 ):
    """a function that embeds a new query and returns the most probable results"""
    embedded_query = ST.encode(query) # embed new query
    scores, retrieved_examples = data.get_nearest_examples( # retrieve results
        "embeddings", embedded_query, # compare our new embedded query with the dataset embeddings
        k=k # get only top k results
    )
    return scores, retrieved_examples

In [ ]:
# TODO: query 정의
query = ""
scores , retrieved_examples = search(query, k=3)

for i, (title, document) in enumerate(zip(retrieved_examples['title'], retrieved_examples['document'])):
  print(f"Retrieved document {i} : \nTitle: {title}\nDocument: {document}")
  print(f"Score : {scores[i]}\n")

## 3-4. Load model

이제 아래 그림에서 1번 retriever 구현은 끝났습니다.

![](https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/RAG_workflow.png)

이제 LLM을 불러와서 2번 reader를 구현해보세요. 사용할 LLM은 `Qwen/Qwen2.5-3B-Instruct`입니다. OOM 문제가 발생한다면 더 가벼운 모델인 `Qwen/Qwen2.5-1.5B-Instruct`로 변경하시는 것을 추천드립니다. 아래 코드를 실행하여 모델을 불러오고, 실제로 텍스트를 생성해봅시다.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-3B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

prompt = "서울의 유명한 관광 코스를 만들어줄래?"
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512,
    use_cache=True,
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

## 3-5. LLM Agent Pipeline

아래 코드에서 구현한 LLM Agent pipeline을 설명하면 다음과 같습니다:

1. **사용자 입력 처리 (Prompt Handling)**: `rag_chatbot` 함수는 사용자가 입력한 `prompt`를 받아들입니다. 이 `prompt`는 사용자가 알고자 하는 질문이나 요청을 의미합니다.

2. **문서 검색 (Document Retrieval)**: 함수의 첫 단계는 `retriever` 함수를 통해 이루어집니다. 이 함수는 입력된 `prompt`와 관련된 정보를 찾기 위해 검색을 수행합니다. 여기서 `k`는 검색할 문서의 수를 의미하며, 기본값은 2로 설정되어 있습니다. `retriever` 함수는 주어진 `prompt`에 가장 관련성이 높은 `k`개의 문서를 데이터베이스에서 검색해 `retrieved_documents`로 반환합니다.

3. **이유 추론 (Reasoning)**: 검색된 문서들은 이제 `reasoner` 함수로 전달됩니다. 이 함수는 주어진 `prompt`와 검색된 문서들을 바탕으로 논리적인 추론을 수행하여, `reasoner_output`을 생성합니다. 이 과정에서, 검색된 정보들을 종합해 사용자 질문에 대한 초기 답변을 생성합니다.

4. **결과 평가 (Evaluation)**: 생성된 초기 답변은 `evaluator` 함수로 전달되어 평가됩니다. 이 단계에서는 `reasoner_output`의 품질이나 정확성을 평가하여, 답변이 얼마나 타당하고 신뢰할 만한지 판단합니다. 평가 결과는 `evaluation_output`으로 반환됩니다.

5. **피드백을 통한 재조정 (Refinement with Feedback)**: 마지막으로, 초기 답변(`reasoner_output`)과 평가 결과(`evaluation_output`)를 바탕으로, `reasoner_with_feedback` 함수가 실행됩니다. 이 함수는 평가 결과를 반영하여 초기 답변을 개선하고, 최종적으로 더 나은 답변을 생성합니다. 이 개선된 최종 답변이 `final_output`으로 반환됩니다.

6. **결과 반환 (Output Return)**: 함수는 `retrieved_documents`, `reasoner_output`, `evaluation_output`, `final_output`의 순서로 네 가지 출력을 반환합니다. 이를 통해 사용자는 검색된 문서, 초기 답변, 평가 결과, 그리고 피드백을 반영한 최종 답변을 모두 확인할 수 있습니다.

이와 같은 흐름을 통해 LLM Agent는 사용자 질문에 대해 관련 정보를 검색하고, 추론하여 답변을 생성하며, 그 답변을 평가하고 개선하는 일련의 과정을 수행합니다. 이로써 최종적으로 사용자에게 신뢰할 수 있고 유용한 답변을 제공하게 됩니다.

In [ ]:
def generate(formatted_prompt):
  formatted_prompt = formatted_prompt[:2000] # to avoid GPU OOM
  messages = [{"role":"user","content":formatted_prompt}]
  # tell the model to generate
  text = tokenizer.apply_chat_template(
      messages,
      tokenize=False,
      add_generation_prompt=True
  )

  model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

  generated_ids = model.generate(
      **model_inputs,
      max_new_tokens=512,
      use_cache=True
  )
  generated_ids = [
      output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
  ]

  response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

  return response

def retriever(query,k):
  scores , retrieved_documents = search(query, k)

  return retrieved_documents

def reasoner(query,retrieved_documents,k):
  """using the retrieved documents we will prompt the model to generate our responses"""
  prompt_with_context = f"정보:"
  for idx in range(k) :
    prompt_with_context+= f"{retrieved_documents['document'][idx]}\n"
  prompt_with_context += f"\n\n질문: {query}"

  print(f"입력: {prompt_with_context}")
  return generate(prompt_with_context)

def evaluator(query, response):
  evaluation_prompt = f"""
  당신은 공정한 평가자 언어 모델입니다.

  평가 기준을 나타내는 점수 루브릭에 따라 지시사항과 평가할 응답이 주어집니다.
  1. 주어진 점수 루브릭에 엄격히 기반하여 응답의 질을 평가하는 자세한 피드백을 작성하세요. 일반적으로 평가하지 마세요.
  2. 피드백을 작성한 후에는 1에서 5 사이의 정수로 점수를 작성하세요. 점수 루브릭을 참조해야 합니다.
  3. 출력 형식은 다음과 같아야 합니다: '피드백: [피드백] [결과] [1에서 5 사이의 정수]'
  4. 다른 서론, 결론 및 설명은 생성하지 마세요. 출력에 [결과]를 반드시 포함하세요.
  5. 간결성은 점수에 반영하지 마세요: 질문을 다루는 정확한 답변은 추가적인 불필요한 정보가 포함되어 있더라도 최고 점수를 받아야 합니다.

  평가할 지시사항:
  {query}

  평가할 응답:
  {response}

  점수 루브릭:
  [응답이 지시사항을 얼마나 잘 따르고 있습니까?]
  점수 1: 응답이 지시사항과 매우 벗어나거나 불완전하며, 중대한 사실적 오류가 있습니다.
  점수 2: 응답이 지시사항과 일부 벗어나거나, 불완전하거나, 소규모 사실적 오류가 있습니다.
  점수 3: 응답이 지시사항을 대부분 따르고 있으나, 약간의 벗어남이나 사소한 오류가 있습니다.
  점수 4: 응답이 지시사항을 잘 따르며, 대부분 완전하고 정확합니다.
  점수 5: 응답이 지시사항을 완벽히 준수하며, 완전하고 정확하며 전적으로 사실적입니다.

  피드백:
  """

  return generate(evaluation_prompt)

def reasoner_with_feedback(query, response, evaluation_output):
  regeneration_prompt = f"""
  당신은 피드백을 기반으로 응답을 재생성하는 언어 모델입니다.

  주어진 지시사항, 원래의 응답, 그리고 해당 응답에 대한 피드백이 제공됩니다. 피드백을 참고하여 개선된 응답을 작성하세요.

  지시사항:
  {query}

  원래의 응답:
  {response}

  피드백:
  {evaluation_output}

  재생성된 응답을 다음과 같은 형식으로 제출하세요:
  '재생성된 응답: '
  """

  return generate(regeneration_prompt)

def rag_chatbot(query:str,k:int=2):
  retrieved_documents = retriever(query, k)
  reasoner_output = reasoner(query,retrieved_documents,k)
  evaluation_output = evaluator(query,reasoner_output)
  final_output = reasoner_with_feedback(query,reasoner_output,evaluation_output)

  return retrieved_documents, reasoner_output, evaluation_output, final_output


In [ ]:
# TODO: query 정의
query = ""
retrieved_documents, reasoner_output, evaluation_output, final_output = rag_chatbot(query, k = 2)

In [ ]:
print(f"Retrieved Documents: \n{retrieved_documents}\n\n")
print(f"Reasoner Output: \n{reasoner_output}\n\n")
print(f"Evaluation Output: \n{evaluation_output}\n\n")
print(f"Final Output: \n{final_output}\n\n")

**검색한 문서들을 기반으로 바로 한 번에 답변하는 대신, 왜 위와 같은 LLM Agent를 구현해서 총 3번에 걸쳐서 최종 답변을 생성했을까요? 이와 같은 Agentic Flow을 사용하는 이유에 대해 본인의 생각을 3줄 이상 적어주세요:**

## 3-6. Your LLM Agent Pipeline

이제 본인만의 LLM Agent를 구현해봅시다. 방법은 여러 가지가 있습니다. **`예를 들어 유저 쿼리를 재구성하여 retrieval에 사용할 수 있습니다.`** 사용자의 질문이 주로 의문문 형식으로 되어 있고, 정답을 포함하는 문서는 평서문 형식으로 되어 있어서 retriever의 성능이 떨어질 수 있습니다. 이러한 경우, 의미적 유사도(Semantic Similarity)를 계산할 때 질문과 문서의 문장 구조가 달라서 실제로 가장 관련성이 높은 문서가 낮은 점수를 받을 수 있습니다. 이로 인해 중요한 정보를 놓칠 위험이 있습니다.

이를 해결하기 위해 다음과 같은 접근 방식을 고려할 수 있습니다:

**Query Expansion and Reformulation (쿼리 확장 및 재구성)**
- **방법**: 사용자 질문을 여러 가지 형태로 확장하거나 재구성하여 검색을 수행합니다. 예를 들어, 의문문을 평서문으로 변환하거나, 질문의 키워드를 추출하여 다양한 형태의 유사 쿼리를 생성합니다. 이를 통해 평서문과 의문문 간의 문장 구조 차이를 완화할 수 있습니다.
- **구현 예시**: "왜 테슬라의 주가가 상승했나요?" → "테슬라의 주가는 ... 때문에 상승했습니다." 또는 "테슬라 주가 상승의 이유".
- **이점**: 문장 구조의 차이로 인해 발생하는 유사도 계산의 오류를 줄이고, 더 관련성 높은 문서를 찾을 수 있습니다.

이외에도 **`retrieve한 문서들을 우선 요약한 다음 최종 답변을 생성할 수도 있고`, `답변의 명확성을 위해 LLM에게 답변 형식과 포함해야될 내용을 지정해줄 수도 있습니다.`** 답변 형식은 예를 들어 다음과 같이 프롬프트에 지정할 수 있습니다:
```
질문: {query}

정보:
{retrieved_documents}

지시사항:
주어진 질문에 대해 아래의 답변 구조를 따라 답변을 작성하세요.
1. 요약: 질문에 대한 핵심 정보를 간략하게 요약합니다. 주어진 뉴스 기사에서 중요한 내용을 추출하여, 간결하고 명확하게 요약하세요.
2. 상세 설명: 뉴스 기사의 주요 내용을 기반으로 세부 정보를 제공합니다. 답변이 충분히 상세하고 구체적인 정보를 포함하도록 하세요.
3. 결론/의견: 제공된 정보를 바탕으로 결론을 내리거나 요약된 내용의 함의를 설명합니다. 이 단계에서는 정보의 의미나 결과를 요약하고, 질문에 대한 종합적인 답변을 제시하세요.

답변:
```

LLM Agent를 구현하는 방식은 다양합니다. 위의 예시들을 참고하여 구현할 수도 있고, 다른 방식으로 구현하셔도 좋습니다. **아래에 본인의 LLM Agent를 구현하고 이에 대한 설명을 간단하게 적어주세요.**

In [ ]:
# TODO: LLM Agent 구현

In [ ]:
# TODO: LLM Agent을 활용한 답변 생성
# 쿼리를 하나 만들어서 실제로 실행을 하고 결과물을 출력해주세요.

**설명:**